# Kubios Cloud demo 2 (Javascript version)

10.2.2024, Sakari Lukkarinen\
Health, Information and Communication Technology\
Metropolia University of Applied Sciences

## Introduction

The aim of this Notebook is to demonstrate how to convert Kubios Cloud demo 2 Python code to JavaScript. Bing Copilot is used to convert the Python code snippets to JavaScript. **Note, the code has not been tested, but it works as starting point for writing your own backend code.**

The original Python code snippets are based on the Kubioscloud example for authorization and Kubios Cloud API reference (see the References).

### How to use this Notebook

- Copy the codes to your own coding environment and modify and verify them.
- For USERNAME and PASSWORD use the Kubios HRV application username and password.
- You can use also the same username and password to login into the Kubios Cloud API.
- In addition, you need CLIENT_ID to read the data from Kubioscloud.
- Some of the following code snippets needs info/data from the previous steps, so check carefully how the information is extracted from the JSON data-structures and entered to the next code.
- At the end you should have all your analysis and raw data written into JSON and CSV files.

### References

- [Kubioscloud example for authorization](https://bitbucket.org/kubios/workspace/snippets/4X95xd/kubioscloud-example-for-authorization-code)
- [Kubios Cloud API reference](https://analysis.kubioscloud.com/v1/portal/documentation/apis.html#kubioscloud-api-reference)

## 1. Setup

Here's how you can import similar libraries in JavaScript:

```javascript
// Python's 'base64' can be replaced with 'Buffer' in Node.js
// No need to import 'logging' in JavaScript, you can use console.log()
// Python's 're' can be replaced with JavaScript's built-in RegExp
// Python's 'uuid' can be replaced with 'uuid' in Node.js
// Python's 'pprint' can be replaced with 'console.log()' in JavaScript
// Python's 'requests' can be replaced with 'axios' in Node.js
// Python's 'matplotlib.pyplot' doesn't have a direct equivalent in JavaScript
// Python's 'numpy' doesn't have a direct equivalent in JavaScript
// Python's 'urllib' can be replaced with 'url' in Node.js
// Python's 'json' can be replaced with JavaScript's built-in JSON

const axios = require('axios');
const uuid = require('uuid');
const url = require('url');
```

Please note that JavaScript (especially in the browser) doesn't have direct equivalents for some Python libraries like `matplotlib` and `numpy`. For complex numerical computations and plotting in JavaScript, you might need to look into specific libraries or use a combination of libraries. Also, the code above is for a Node.js environment, and it may vary if you're working in a different JavaScript environment. 

Remember to install the necessary npm packages if you're working in a Node.js environment. You can do this by running `npm install <package-name>` in your terminal. For example, `npm install axios uuid url`. 

Also, please note that the conversion of code from one language to another is not always straightforward and may require understanding of the logic and functionality you want to implement. The provided JavaScript code is just for importing similar libraries and may not work exactly the same way as in Python. It's always recommended to understand the functionality of each library and find the best equivalent in the target language.

### Username, password, and client_id

```javascript
// Use your Kubios HRV App username and password
const USERNAME = "...";
const PASSWORD = "...";

// Client ID is given in a separate document found from OMA workspace
const CLIENT_ID = "...";

const LOGIN_URL = "https://kubioscloud.auth.eu-west-1.amazoncognito.com/login";
const TOKEN_URL = "https://kubioscloud.auth.eu-west-1.amazoncognito.com/oauth2/token";
const REDIRECT_URI = "https://analysis.kubioscloud.com/v1/portal/login";

const USER_AGENT = "Hyte 2022";  // FIXME: Use unique name for your application
```

Please note that JavaScript uses `const` to define a constant. The value of a constant can't be changed through reassignment, and it can't be redeclared. In JavaScript, all the constants are block-scoped, they can only be accessed within the block they were declared. If you need to change the value later in your code, you can use `let` instead of `const`. Also, JavaScript uses semicolons at the end of each statement.

## 2. Opening a session and get tokens

Here's how you can open a session ang get tokens in JavaScript:

```javascript
// Import required modules
const axios = require('axios');
const uuid = require('uuid');
const url = require('url');

// Define constants
const USERNAME = "...";
const PASSWORD = "...";
const CLIENT_ID = "...";
const LOGIN_URL = "https://kubioscloud.auth.eu-west-1.amazoncognito.com/login";
const TOKEN_URL = "https://kubioscloud.auth.eu-west-1.amazoncognito.com/oauth2/token";
const REDIRECT_URI = "https://analysis.kubioscloud.com/v1/portal/login";
const USER_AGENT = "Hyte 2022";

// Generate CSRF token
let csrf = uuid.v4();

// Define login data
let login_data = {
    client_id: CLIENT_ID,
    redirect_uri: REDIRECT_URI,
    username: USERNAME,
    password: PASSWORD,
    response_type: "code",
    access_type: "offline",
    _csrf: csrf,
};

// Log info
console.log(`Authenticating to '${LOGIN_URL}' with client_id: ${CLIENT_ID}`);

// Make a POST request to login
axios.post(LOGIN_URL, login_data, {
    headers: {
        "Cookie": `XSRF-TOKEN=${csrf}`,
        "User-Agent": USER_AGENT
    },
    maxRedirects: 0
}).then((login_response) => {
    // Check status code
    if (login_response.status !== 302) {
        throw new Error(`Status: ${login_response.status}, Authentication failed.`);
    }

    // Get the code
    let code = url.parse(login_response.headers.location, true).query.code;
    console.log(`Got code: ${code}`);

    // Define exchange data
    let exch_data = {
        client_id: CLIENT_ID,
        code: code,
        redirect_uri: REDIRECT_URI,
        grant_type: "authorization_code",
    };

    // Log info
    console.log("Exchanging code to tokens");

    // Make a POST request to exchange tokens
    return axios.post(TOKEN_URL, exch_data);
}).then((exch_response) => {
    // Log status code
    console.log(`Status code ${exch_response.status}`);

    // Get tokens
    let tokens = exch_response.data;

    // Log keys
    console.log(Object.keys(tokens));

    // Log token expiry
    console.log(`Tokens expires in: ${tokens.expires_in} seconds.`);

    // Log id_token
    console.log(`id_token: ${tokens.id_token}`);
}).catch((error) => {
    console.error(error);
});
```

Please note that this JavaScript code is for a Node.js environment and uses the `axios` library for HTTP requests. You'll need to install `axios` and `uuid` using npm by running `npm install axios uuid` in your terminal. Also, error handling in JavaScript is typically done with `.catch()` at the end of a promise chain, which will catch any errors that occur in any of the promises. This is different from Python's `assert` statement. 
- The `url` module is used to parse the URL and get the query parameters. This is similar to splitting the string in Python. 
- The `console.log()` function is used for logging, similar to the `logging` module in Python. 
- The `v4()` method from the `uuid` module is used to generate a random UUID, similar to `uuid.uuid4()` in Python. 
- The `Object.keys()` function is used to get the keys of an object, similar to the `keys()` method in Python. 
- The `data` property of the response from `axios` is used to get the response data, similar to the `json()` method in Python. 
- The `status` property of the response from `axios` is used to get the status code, similar to the `status_code` property in Python. 
- The headers are set in the options object in `axios`, similar to the `headers` parameter in Python's `requests`. 
- The `maxRedirects: 0` option is used to prevent redirects, similar to `allow_redirects=False` in Python's `requests`. 
- The `post()` method from `axios` is used to make a POST request, similar to the `post()` method from Python's `requests`. 
- The `then()` method is used to handle the promise returned by `axios.post()`, similar to the assignment of the response to a variable in Python. 
- The `catch()` method is used to handle any errors that occur in the promise chain, similar to the `assert` statement in Python. 
- The `throw new Error()` statement is used to throw an error, similar to raising an exception in Python. 
- The `url.parse()` function is used to parse the URL and get the query parameters, similar to splitting the string in Python. 
- The `console.error()` function is used to log errors, similar to the `logging.error()` function in Python. 
- The `require()` function is used to import modules, similar to the `import` statement in Python. 

## 3. Get user info and results
For more details, see:

- [KubiosCloud API reference](https://analysis.kubioscloud.com/v2/portal/documentation/apis.html#kubioscloud-api-reference)
- [Get user information](https://analysis.kubioscloud.com/v2/portal/documentation/apis.html#get-user-information)
- [Get results](https://analysis.kubioscloud.com/v2/portal/documentation/apis.html#get-results)
    - For daily readiness results, see the example at the end "Get results"

**NOTE:** You need to create a subfolder json where all collected data will be stored.

Here's how you can get user info and measurement results with JavaScript:

```javascript
// Import required modules
const axios = require('axios');
const fs = require('fs');

// Define constants
const USERNAME = "...";
const PASSWORD = "...";
const CLIENT_ID = "...";
const USER_AGENT = "Hyte 2022";
const TOKEN_URL = "https://kubioscloud.auth.eu-west-1.amazoncognito.com/oauth2/token";
const ID_TOKEN = "..."; // Replace with the actual id_token

// Create headers
let HEADERS = {"Authorization": ID_TOKEN, "User-Agent": USER_AGENT};

// Create URLs
const BASE_URL = "https://analysis.kubioscloud.com";
const GET_USER_INFO = BASE_URL + "/v2/user/self";
const GET_RESULT = BASE_URL + "/v2/result/self?from=2020-01-01T00%3A00%3A00%2B00%3A00";
const GET_DAILY_READINESS = BASE_URL + "/v2/result/self?types=readiness&daily=yes";

// Return personal information for the currently authenticated user
console.log("Get user info");
axios.get(GET_USER_INFO, {headers: HEADERS}).then((response) => {
    let user_info = response.data;
    fs.writeFileSync('./json/user_info.json', JSON.stringify(user_info));
    console.log("Wrote user info to JSON file.");
});

// List all results for the current user from the last 30 days (all defaults)
console.log("Get results");
axios.get(GET_RESULT, {headers: HEADERS}).then((response) => {
    let all_results = response.data;
    fs.writeFileSync('./json/all_results.json', JSON.stringify(all_results));
    console.log("Wrote results to JSON file.");
});

// List only “daily” readiness results
console.log("Get daily readiness results");
axios.get(GET_DAILY_READINESS, {headers: HEADERS}).then((response) => {
    let daily_readiness = response.data;
    fs.writeFileSync('./json/daily_readiness.json', JSON.stringify(daily_readiness));
    console.log("Wrote daily readiness results to JSON file.");
}).catch((error) => {
    console.error(error);
});
```

Please note that this JavaScript code is for a Node.js environment and uses the `axios` library for HTTP requests. You'll need to install `axios` using npm by running `npm install axios` in your terminal. Also, error handling in JavaScript is typically done with `.catch()` at the end of a promise chain, which will catch any errors that occur in any of the promises. This is different from Python's `assert` statement. 
- The `fs.writeFileSync()` function is used to write data to a file, similar to opening a file in write mode and writing to it in Python. 
- The `JSON.stringify()` function is used to convert a JavaScript object into a JSON string, similar to the `json.dump()` function in Python. 
- The `console.log()` function is used for logging, similar to the `logging` module in Python. 
- The `console.error()` function is used to log errors, similar to the `logging.error()` function in Python. 
- The `require()` function is used to import modules, similar to the `import` statement in Python.

## More info

Here are other generic notes on tranlation from Python to JavaScript:

- The url module is used to parse the URL and get the query parameters. This is similar to splitting the string in Python. 
- The console.log() function is used for logging, similar to the logging module in Python. 
- The v4() method from the uuid module is used to generate a random UUID, similar to uuid.uuid4() in Python. 
- The Object.keys() function is used to get the keys of an object, similar to the keys() method in Python. The data property of the response from axios is used to get the response data, similar to the json() method in Python. 
- The status property of the response from axios is used to get the status code, similar to the status_code property in Python. The headers are set in the options object in axios, similar to the headers parameter in Python’s requests. 
- The maxRedirects: 0 option is used to prevent redirects, similar to allow_redirects=False in Python’s requests. 
- The post() method from axios is used to make a POST request, similar to the post() method from Python’s requests. 
- The then() method is used to handle the promise returned by axios.post(), similar to the assignment of the response to a variable in Python. 
- The catch() method is used to handle any errors that occur in the promise chain, similar to the assert statement in Python. 
- The throw new Error() statement is used to throw an error, similar to raising an exception in Python. 
- The url.parse() function is used to parse the URL and get the query parameters, similar to splitting the string in Python. 
- The console.error() function is used to log errors, similar to the logging.error() function in Python. 
- The require() function is used to import modules, similar to the import statement in Python. 

- The const keyword is used to declare constants, similar to the assignment of constants in Python. 
- The let keyword is used to declare variables that can be reassigned, similar to the assignment of variables in Python. 
- The = operator is used for assignment, similar to the = operator in Python. 
- The : operator is used to define properties in an object, similar to the : operator in Python. 
- The {} syntax is used to define an object, similar to the {} syntax in Python. 
- The . operator is used to access properties of an object, similar to the . operator in Python. 
- The ; character is used to end statements, similar to the newline character in Python. 
- The () syntax is used to call functions, similar to the () syntax in Python. 
- The => syntax is used to define arrow functions, similar to the def keyword in Python. 
- The {} syntax is used to define a block of code, similar to the indentation in Python. 
- The if keyword is used for conditional statements, similar to the if keyword in Python. 

- The new keyword is used to create a new instance of a class, similar to calling a class in Python. 
- The Error class is used to create a new error, similar to the Exception class in Python. 
- The return keyword is used to return a value from a function, similar to the return keyword in Python. 
- The , character is used to separate items in a list, similar to the , character in Python. 
- The [] syntax is used to access elements in an array, similar to the [] syntax in Python. 
- The + operator is used for string concatenation, similar to the + operator in Python. 
- The " character is used to define strings, similar to the " character in Python. 
- The : character is used to separate the key and value in an object, similar to the : character in Python. 
- The , character is used to separate properties in an object, similar to the , character in Python. 
- The " character is used to define keys in an object, similar to the " character in Python. The " character is used to define values in an object, similar to the " character in Python. 
- The {} syntax is used to define an empty object, similar to the {} syntax in Python. 
- The = operator is used to assign a value to a variable, similar to the = operator in Python. 
The . operator is used to access methods of an object, similar to the . operator in Python. 
- The () syntax is used to call a method, similar to the () syntax in Python.